In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd
import leafmap as leafmap

In [6]:
geom1_gdf = gpd.read_file('data/terrawise/Stratification/1/cea part/cea_1.shp')
geom2_gdf = gpd.read_file('data/terrawise/Stratification/2/cea part/cea_2.shp')
geom3_gdf = gpd.read_file('data/terrawise/Stratification/3/cea part/cea_3.shp')

geom1_gdf.head()


,FID,geometry
0,0,"POLYGON ((612745.937 6274514.427, 613156.219 6..."
1,1,"POLYGON ((613969.669 6274500.797, 613964.708 6..."


In [7]:
"""
label - assume 123 correspond to cea 1,2,3
year - will need to be able to filter by year
For a given year, map should show areas where a practice was/ was not implemented.
"""
practice_csv = pd.read_csv('data/terrawise/terrawise-practice-info.csv')
practice_csv.head()

,label,Year,Land use,Knifepoint_seeder,Disc_seeder,Cover_crop,VRT,Open_front_harvester,Stripper_front_harvester,Stubble_less15,...,Rotational_graze,Early_crop_graze,Groundcover_measure,FOO_monitor,Sow_summer_fodder,Early_sale,Feedlot,Early_wean,Livestock_tech,Tech
0,0,2022,Crop,Y,N,N,N,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2021,Crop,Y,N,N,N,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2020,Crop,Y,N,N,N,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2019,Crop,Y,N,N,N,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2018,Crop,Y,N,N,N,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
